# Sprint25課題 Seq2Seq

## 【問題1】機械翻訳の実行とコードリーディング
Keras公式のサンプルコードで、短い英語からフランス語への変換が行えるのでこれを動かしてください。

[keras/lstm_seq2seq.py at master · keras-team/keras](https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py)

その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめてください。以下のようにどこからどこの行が何をしているかを記述してください。

（例）

- 51から55行目 : ライブラリのimport
- 57から62行目 : ハイパーパラメータの設定

In [1]:
# ライブラリのインポート
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


Using TensorFlow backend.


In [2]:
# ハイパーパラメータの設定
batch_size = 64  # 学習時のバッチサイズ
epochs = 5  # 学習実のエポック数
latent_dim = 256  # エンコーダの隠れ層の次元
num_samples = 10000  # 学習データのサンプル数

# データのパス
data_path = 'fra-eng/fra.txt'

# データをベクトル化する
# テキスト格納用のlist, taple
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# データの読み込み
# [英語 フランス語]の単語、文章
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
# num_sampleまたはデータ数に応じて学習を行う
for line in lines[: min(num_samples, len(lines) - 1)]:
    
    #英語をinput_textへ、フランス語をtarget_textへ
    input_text, target_text = line.split('\t')
    
    # target_textに対して、start sequenceとしてtab('\t')、
    #  "end sequence" として"\n"を使用する。
    target_text = '\t' + target_text + '\n'
    
    #listに格納する
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # textからユニーク文字列をリストに格納していく
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# 文字列リストをソート
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# 文字列の数をエンコーダ、デコーダーのtokenにする
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# textの中で一番大きい文字数を測る
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#　いろいろとプリント
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 文字列にインデックスを付与する
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# エンコーダ入力データ、デコーダー入力データ・ターゲットデータの初期化
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# 入力テキストとターゲットテキスト
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # テキスト内の文字列と一致するインデックスを1にする
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_dataはdecoder_input_dataより1ステップさきに進んでいる
        decoder_input_data[i, t, target_token_index[char]] = 1.
        
        if t > 0:
            # ここでdecoder_target_dataが1ステップ先に進み、
            # startの文字列は含まないようにする
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# 入力データのシーケンスを定義して、エンコーディング処理を行う
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# `encoder_outputs`を破棄し、encoder_statesのみを保持する。
encoder_states = [state_h, state_c]

# 初期状態として `encoder_states`を使ってデコーダーを設定する。
decoder_inputs = Input(shape=(None, num_decoder_tokens))


# 内部statesを返すのと同様、出力シーケンスを返したデコーダーを設定する
# この戻り値は学習モデルには使用しないが、推論時に使用する。
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# `encoder_input_data`と` decoder_input_data`を 
# `decoder_target_data`に変換するモデルを定義
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 学習
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

# モデルを保存
model.save('s2s.h5')

# Next：推論モード（サンプリング）
#1）入力をエンコードしてstatesを取得する
#2）statesとターゲットの"start of sequence"トークンでデコーダを1ステップ実行する
#　　　出力は次のターゲットトークンになる
#3）現在のターゲットトークンと現在の状態で繰り返す

# サンプリングモデルの定義をする
encoder_model = Model(encoder_inputs, encoder_states)

# 隠れ層の次元数分の入力を設定する(hとCの分)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# デコーダの入力とそのstatesより、デコーダーのstatesを設定する
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

# モデルに渡す
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 時系列データを読み取り可能なものにするための逆引きトークンインデックス。
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())



Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 62s 8ms/step - loss: 0.9216 - val_loss: 0.9535
Epoch 2/5
8000/8000 [==============================] - 57s 7ms/step - loss: 0.7366 - val_loss: 0.7659
Epoch 3/5
8000/8000 [==============================] - 59s 7ms/step - loss: 0.6223 - val_loss: 0.6807
Epoch 4/5
8000/8000 [==============================] - 57s 7ms/step - loss: 0.5645 - val_loss: 0.6366
Epoch 5/5
8000/8000 [==============================] - 57s 7ms/step - loss: 0.5243 - val_loss: 0.5934


/Users/yusuke-saruya/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [3]:
def decode_sequence(input_seq):
    # 入力を状態ベクトルとしてエンコードする
    states_value = encoder_model.predict(input_seq)

    # 長さ1の空のターゲット時系列データを生成
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # ターゲットシーケンスの最初の文字に開始文字'\t'を入力
    target_seq[0, 0, target_token_index['\t']] = 1.

    # シーケンスのバッチ数分サンプルデータをループする
    # (簡単にするためにここでは、サイズ1のバッチを想定する)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        #ターゲットシーケンスとエンコードしたstateからデコードする
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # トークンをサンプリング
        #デコードした状態から最大値のインデックスを取得
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        # 逆引きしたトークンのインデックスから文字を取得
        sampled_char = reverse_target_char_index[sampled_token_index]
        
        # デコードされた文章に追加する
        decoded_sentence += sampled_char

        # 文の文字数が最大に達するか、または停止文字があった場合、ループ停止
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 長さ1のターゲットシーケンスを更新する
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # statesを更新する
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # デコーディングを試すために学習データの一部からシーケンスを一つ取り出す
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Arrête !

-
Input sentence: Hi.
Decoded sentence: Arrête !

-
Input sentence: Run!
Decoded sentence: Arrête !

-
Input sentence: Run!
Decoded sentence: Arrête !

-
Input sentence: Who?
Decoded sentence: Qui est chait ?

-
Input sentence: Wow!
Decoded sentence: Sois cous en cherte ?

-
Input sentence: Fire!
Decoded sentence: Arrête !

-
Input sentence: Help!
Decoded sentence: Arrête !

-
Input sentence: Jump.
Decoded sentence: Artene le moit !

-
Input sentence: Stop!
Decoded sentence: Arrête !

-
Input sentence: Stop!
Decoded sentence: Arrête !

-
Input sentence: Stop!
Decoded sentence: Arrête !

-
Input sentence: Wait!
Decoded sentence: At chent !

-
Input sentence: Wait!
Decoded sentence: At chent !

-
Input sentence: Go on.
Decoded sentence: Atrenez l'ent la !

-
Input sentence: Go on.
Decoded sentence: Atrenez l'ent la !

-
Input sentence: Go on.
Decoded sentence: Atrenez l'ent la !

-
Input sentence: Hello!
Decoded sentence: Arrête !

-
Inpu

## 3.イメージキャプショニング
他の活用例としてイメージキャプショニングがあります。画像に対する説明の文章を推定するタスクです。これは画像を入力し、時系列を出力するImage to Sequenceの手法によって行えます。

[pytorch-tutorial/tutorials/03-advanced/image_captioning at master · yunjey/pytorch-tutorial](https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/03-advanced/image_captioning)

イメージキャプショニングは学習に多くの時間がかかるため、ここでは学習済みの重みが公開されている実装を動かすことにします。Kerasには平易に扱える実装が公開されていないため、今回はPyTorchによる実装を扱います。

## 【問題2】イメージキャプショニングの学習済みモデルの実行
上記実装において5. Test the modelの項目を実行してください。また、自身で用意した画像に対しても文章を生成してください。これらに対してどういった文章が出力されたかを記録して提出してください。

データセットからの学習は行わず、学習済みの重みをダウンロードして利用します。

注意点として、デフォルトで設定されている重みのファイル名と、ダウンロードできる重みのファイル名は異なっています。ここは書き換える必要があります。



- example  
![example](png/example.png)

In [4]:
!python sample.py --image='png/example.png'

<start> a group of giraffes standing next to each other . <end>


- testimage
![testimage](png/testimage.png)

In [5]:
!python sample.py --image='png/testimage.png'

<start> a group of people sitting around a table with a cake . <end>


自身で用意した画像に対しても、大勢の人、ケーキのおもちゃを捉えるなど、おおよその状況を表した文章を返すことができた。

## 【問題3】Kerasで動かしたい場合はどうするかを調査
PyTorchによる実装を動かしましたが、何らかの理由からKerasで動かしたい状況が考えられます。どういった手順を踏むことになるか調査し、できるだけ詳しく説明してください。

特に今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点については必ず触れてください。

### １．学習済みモデルをPyTorchからKerasに変換させる
- MMdnn(Microsoft Researchにより開発が進められているオープンソースの深層学習モデルの変換と可視化を行うツール)を利用して学習済みモデルの変換を行う。その際、MMdnnではアーキテクチャと重みが含まれるモデルを変換することができるが、重みしか含まれていないモデルは変換することができない。"AttributeError: 'collections.OrderedDict' object has no attribute 'state_dict'"

### ２．コードの書き換え
- PyTorchにより実装されているモデル構築から推測までのコードをKerasを利用して書き換える。


## 【問題4】（アドバンス課題）コードリーディングと書き換え
モデル部分はmodel.pyに書かれていますが、Kerasではこのモデルがどのように記述できるかを考え、コーディングしてください。その際機械翻訳のサンプルコードが参考になります。

**※省略**

## 【問題5】（アドバンス課題）発展的調査
**他の言語の翻訳を行う場合は？**

問題1の実装を使い日本語と英語の翻訳を行いたい場合はどのような手順を踏むか考えてみましょう。

**機械翻訳の発展的手法にはどのようなものがある？**

機械翻訳のための発展的手法にはどういったものがあるか調査してみましょう。

**文章から画像生成するには？**

イメージキャプショニングとは逆に文章から画像を生成する手法もあります。どういったものがあるか調査してみましょう。

**※省略**